# Esercizio


In [52]:
import numpy as np
import scipy.sparse as sp
import gurobipy as gp
from gurobipy import GRB

# Dimensioni scacchiera:
n = 256

# Creazione modello:
m = gp.Model("NQUEENS")

# Introduciamo le variabili decisionali:
x = m.addMVar((n, n), vtype=GRB.BINARY, name="x")

# Funzione Obiettivo


In [53]:
# Funzione obiettivo: massimizzare il numero delle regine presenti (n):
m.setObjective(x.sum(), GRB.MAXIMIZE)

# Vincoli (versione 1)


In [54]:

# Per ogni riga/colonna valgono i seguenti constraints:
for i in range(n):

    # Al massimo una regina per riga:
    m.addConstr(x[i, :].sum() <= 1, name="row"+str(i))

    # Al massimo una regina per colonna:
    m.addConstr(x[:, i].sum() <= 1, name="col"+str(i))

# Per ogni diagonale devono valere i seguenti constraints: (sono 2n-1 diagonali)
for k in range(1, 2*n):

    # Al massimo una regina per ogni diagonale:

    # NB: Elenco gli indici delle diagonali: (da dx in basso a sx in alto)
    # righe i:   -  colonne j:
    # range(0,1) ,  range(1,0).
    # range(0,2) ,  range(2,0).
    # range(0,3) ,  range(3,0)...
    #(range parte da sx e arriva a dx-1)
    
    diagn = (range(max(0, k-n), min(n, k)), range(min(n, k)-1, max(0, k-n)-1, -1))
    m.addConstr(x[diagn].sum() <= 1, name="diag"+str(k))

    # Al massimo una regina per ogni anti-diagonale:


    # NB: Elenco gli indici delle anti-diagonali: (sx in basso a dx in alto)
    # righe i:   -  colonne j:
    # range(0,1) ,  range(7,8).
    # range(0,2) ,  range(6,8).
    # range(0,3) ,  range(5,8)...
    adiagn = (range(max(0, k-n), min(n, k)), range(max(0, n-k), min(n, 2*n-k)))
    m.addConstr(x[adiagn].sum() <= 1, name="adiag"+str(k))


# Vincoli (versione 2)

In [55]:
# Per ogni riga/colonna valgono i seguenti constraints:
for i in range(n):

    # Al massimo una regina per riga:
    m.addConstr(x[i, :].sum() <= 1, name="row"+str(i))

    # Al massimo una regina per colonna:
    m.addConstr(x[:, i].sum() <= 1, name="col"+str(i))

# Vincolo diag:
# Elenco tutti gli indici da considerare per ogni diag nella somma (vettori I e J)

for k in range(1-n,n-1):
    I = [] 
    J = []
    for i in range(n):
        for j in range(n):
            if (i-j)==k:
                I.append(i)
                J.append(j)
    m.addConstr(x[I,J].sum() <= 1, name="diag"+str(k))
    

# Vincolo adiag:
# Elenco tutti gli indici da considerare per ogni adiag nella somma (vettori I e J)

for k in range(2,2*n):
    I = [] 
    J = []
    for i in range(n):
        for j in range(n):
            if (i+j)==k:
                I.append(i)
                J.append(j)
    m.addConstr(x[I,J].sum() <= 1, name="adiag"+str(k))


In [56]:
# Ottimizzazione
m.optimize()

print(x.X)
print('Queens placed: %g' % m.ObjVal)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 90 rows, 64 columns and 508 nonzeros
Model fingerprint: 0x28c0a1c5
Variable types: 0 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 5.0000000
Presolve removed 48 rows and 0 columns
Presolve time: 0.00s
Presolved: 42 rows, 64 columns, 269 nonzeros
Variable types: 0 continuous, 64 integer (64 binary)

Root relaxation: objective 8.000000e+00, 52 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                       8.0000000   64.00000   700%     -    0s
     0     0          -    0         8.00000    8.00000  0.00%    

In [57]:
# Rappresentazione grafica del risultato ottenuto:

GRID_SIZE = n
ris = x.X

def create_grid():
    """Creates the grid return=an array that represents the grid"""

    tab = []
    for i in range(GRID_SIZE):
        tab.append([])
        for j in range(GRID_SIZE):
            tab[i].append("")
    return tab
    
def print_grid(tab):
    """prints the grid"""

    for i in range(GRID_SIZE):
        print("{} ".format(i + 1), end="")
        for j in range(GRID_SIZE):
              
            if ris[i][j] == 1:
              print("\u2655{}".format(tab[i][j]), end="")
            else:
              print("--".format(tab[i][j]), end="")
            if j < GRID_SIZE:
                print("|", end="")

        if i < GRID_SIZE:
            print()

In [58]:

#ricorda che le regine possono essere piazzate sulle colonne da 0 a N-1
print("Soluzione Ottenuta: ")
print("Dispsizione scacchiera:")
print_grid(create_grid())

Soluzione Ottenuta: 
Dispsizione scacchiera:
1 --|--|--|--|♕|--|--|--|
2 --|♕|--|--|--|--|--|--|
3 --|--|--|♕|--|--|--|--|
4 --|--|--|--|--|--|♕|--|
5 --|--|♕|--|--|--|--|--|
6 --|--|--|--|--|--|--|♕|
7 --|--|--|--|--|♕|--|--|
8 ♕|--|--|--|--|--|--|--|
